In [11]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
import os

In [18]:
path_to_datasets = '../../proj/gendata'
allnames = os.listdir(path_to_datasets)
allnames.sort()
filenames = []
for name in allnames:
    if name[3] == 'B':
        filenames.append(name)
filenames.sort()
print(filenames)
print(len(filenames))

['SS-B.csv_100_11514.csv', 'SS-B.csv_100_19953.csv', 'SS-B.csv_100_3545.csv', 'SS-B.csv_100_39658.csv', 'SS-B.csv_100_40749.csv', 'SS-B.csv_100_44845.csv', 'SS-B.csv_100_51176.csv', 'SS-B.csv_100_58612.csv', 'SS-B.csv_100_61367.csv', 'SS-B.csv_100_65388.csv', 'SS-B.csv_100_65444.csv', 'SS-B.csv_100_66946.csv', 'SS-B.csv_100_69024.csv', 'SS-B.csv_100_70006.csv', 'SS-B.csv_100_70292.csv', 'SS-B.csv_100_77340.csv', 'SS-B.csv_100_77845.csv', 'SS-B.csv_100_83686.csv', 'SS-B.csv_100_84967.csv', 'SS-B.csv_100_86331.csv', 'SS-B.csv_10_11514.csv', 'SS-B.csv_10_19953.csv', 'SS-B.csv_10_3545.csv', 'SS-B.csv_10_39658.csv', 'SS-B.csv_10_40749.csv', 'SS-B.csv_10_44845.csv', 'SS-B.csv_10_51176.csv', 'SS-B.csv_10_58612.csv', 'SS-B.csv_10_61367.csv', 'SS-B.csv_10_65388.csv', 'SS-B.csv_10_65444.csv', 'SS-B.csv_10_66946.csv', 'SS-B.csv_10_69024.csv', 'SS-B.csv_10_70006.csv', 'SS-B.csv_10_70292.csv', 'SS-B.csv_10_77340.csv', 'SS-B.csv_10_77845.csv', 'SS-B.csv_10_83686.csv', 'SS-B.csv_10_84967.csv', 'SS-B.

In [19]:
def normalize_values(min_val, max_val, value):
    return (value - min_val) / (max_val - min_val)

In [21]:
for name in filenames:
    path = path_to_datasets + '/' + name
    try:
        data = pd.read_csv(path)
    except Exception as error:
        print("exception Occured here ! ", name, )
        continue

    #x columns do not end in !, +, -
    #y columns end in !, +, -
    X_columns = [col for col in data.columns if not (col.endswith('+') or col.endswith('-') or col.endswith('!'))]
    y_columns = [col for col in data.columns if col.endswith('+') or col.endswith('-') or col.endswith('!')]

    X = data[X_columns]
    y = data[y_columns]

    #replacing any ? with nan
    X.replace('?', np.nan, inplace=True)

    #getting the column names for future use
    column_names = X.columns.tolist()
    y_column_names = y.columns.tolist()
    features = y_column_names

    centroids_data = []

    #using minmax scaler to retain original values
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)

    #nan values are replaced with the mean of the column
    imputer = SimpleImputer(strategy='mean')
    X = imputer.fit_transform(X)

    #dbscan model
    dbscan = DBSCAN(eps=0.1, min_samples=3)
    dbscan.fit(X)

    #generating cluster labels
    labels = dbscan.labels_
    data['Cluster'] = labels

    unique_labels = np.unique(labels)
    
    for label in unique_labels:
        if label == -1:
            continue  # Skip noise points
        cluster_size = np.sum(labels == label)
        cluster_centroid = np.mean(X[labels == label], axis=0)
        
        #getting back original values 
        cluster_centroid_original_scale = scaler.inverse_transform([cluster_centroid])
        
        cluster_centroid_formatted = [f"{value:.2f}" for value in cluster_centroid_original_scale[0]]
        
        y_centroid = np.mean(y.values[labels == label], axis=0)
        y_centroid_original_scale = y_centroid  # Assuming y values are not scaled
        y_centroid_formatted = [f"{value:.2f}" for value in y_centroid_original_scale]

        #zipping centroid values of x and y columns together
        centroid_data = {'Cluster': label, **dict(zip(column_names, cluster_centroid_formatted)), 
                         **dict(zip(y_column_names, y_centroid_formatted))}
        
        centroids_data.append(centroid_data)

    centroids_df = pd.DataFrame(centroids_data)
    #print(centroids_df)

    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    #print(f'Number of clusters formed: {n_clusters}')

    #converting strings to numeric
    centroids_df = centroids_df.apply(pd.to_numeric, errors='ignore')

    maxValues = data.max()
    minValues = data.min()

    d2h = []
    for index, centroid in centroids_df.iterrows():
        d = 0
        for column in features:
            if column.endswith("+"):
                d += (abs(1.0 - normalize_values(minValues[column], maxValues[column], centroid[column])))**2
            else:
                d += (abs(0.0 - normalize_values(minValues[column], maxValues[column], centroid[column])))**2
        d2h.append((d/len(features))**(1/2))
    
    d2h = [round(value, 2) for value in d2h]
    centroids_df['d2h'] = d2h
    
    #Print the centroids DataFrame
    #print(centroids_df)
    result_name = 'SS-B-corrected/' + name[:-4] + '-centroids.csv'
    centroids_df.to_csv(result_name,index=False)

    #print("Min d2h: ",min(d2h))

exception Occured here !  SS-B.csv_100_11514.csv
exception Occured here !  SS-B.csv_100_19953.csv
exception Occured here !  SS-B.csv_100_3545.csv
exception Occured here !  SS-B.csv_100_39658.csv
exception Occured here !  SS-B.csv_100_40749.csv
exception Occured here !  SS-B.csv_100_44845.csv
exception Occured here !  SS-B.csv_100_51176.csv
exception Occured here !  SS-B.csv_100_58612.csv
exception Occured here !  SS-B.csv_100_61367.csv
exception Occured here !  SS-B.csv_100_65388.csv
exception Occured here !  SS-B.csv_100_65444.csv
exception Occured here !  SS-B.csv_100_66946.csv
exception Occured here !  SS-B.csv_100_69024.csv
exception Occured here !  SS-B.csv_100_70006.csv
exception Occured here !  SS-B.csv_100_70292.csv
exception Occured here !  SS-B.csv_100_77340.csv
exception Occured here !  SS-B.csv_100_77845.csv
exception Occured here !  SS-B.csv_100_83686.csv
exception Occured here !  SS-B.csv_100_84967.csv
exception Occured here !  SS-B.csv_100_86331.csv
exception Occured her

/var/folders/mf/pc621dwd2z3_jmf5p7t1zn2r0000gn/T/ipykernel_43343/4192977942.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace('?', np.nan, inplace=True)
/var/folders/mf/pc621dwd2z3_jmf5p7t1zn2r0000gn/T/ipykernel_43343/4192977942.py:73: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  centroids_df = centroids_df.apply(pd.to_numeric, errors='ignore')
/var/folders/mf/pc621dwd2z3_jmf5p7t1zn2r0000gn/T/ipykernel_43343/4192977942.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.replace('?', np.n